# Project 2: Red vs. Blue Encryption Method

**SUMMARY:** In this project we will be setting up the device to take in different plain text inputs, the device will encrypt these inputs and we will read back the encrypted text.

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWNANO'
CRYPTO_TARGET = 'NONE'
SS_VER = 'SS_VER_2_1'

%run "Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [2]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd hardware/victims/firmware/simpleserial-base
make clean
make PLATFORM='CWNANO' CRYPTO_TARGET=$2 SS_VER=$3 -j

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1


.././hal/Makefile.hal:250: *** Invalid or empty PLATFORM: NONE. Known platforms: 
+-------------------------------------------------------+
| PLATFORM      | DESCRIPTION                           |
+=======================================================+
| AVR/XMEGA Targets (8-Bit RISC)                        |
+=======================================================+
+-------------------------------------------------------+
| CWLITEXMEGA   | CW-Lite XMEGA (Alias for CW303)       |
+-------------------------------------------------------+
| CW301_AVR     | Multi-Target Board, AVR Target        |
+-------------------------------------------------------+
| CW303         | XMEGA Target (CWLite), Also works     |
|               | for CW308T-XMEGA                      |
+-------------------------------------------------------+
| CW304         | ATMega328P (NOTDUINO), Also works     |
|               | for CW308T-AVR                        |
+-----------------------------------------------

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
arm-none-eabi-gcc (GNU Arm Embedded Toolchain 10-2020-q4-major) 10.2.1 20201103 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Welcome to another exciting ChipWhisperer target build!!
.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CWNANO/stm32f0_startup.lst -I.././simpleserial/ -I.././simpleserial/ -I.././hal -I.././hal/stm32f0 -I.././hal/stm32f0/CMSIS -I.././hal/stm32f0/CMSIS/core -I.././hal/stm32f0/CMSIS/device -I.././hal/stm32f0/Legacy .././hal/stm32f0/stm32f0_startup.S -o objdir-CWNANO/stm3

In [3]:
cw.program_target(scope, prog, "hardware/victims/firmware/simpleserial-base/simpleserial-base-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 14439 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 14439 bytes


In [4]:
from tqdm import tnrange
import aes
import numpy as np
import time

def transpose_matrix(lst):
    matrix = [lst[i:i + 4] for i in range(0, len(lst), 4)]
    transposed = [[matrix[j][i] for j in range(4)] for i in range(4)]
    return [elem for row in transposed for elem in row]

#Enter the system key once per program. If you want to set another key, you will have to re-program the board above

key_input = input("Enter a 16 character Key")
ky = bytearray(key_input.encode('utf-8'))

while len(ky) % 16 != 0:
    ky.append(0)
#transpose matrix for internal AES implmentation
ky_transpose = transpose_matrix(ky)
print("Set the Key to: ", [hex(num) for num in ky_transpose])
cipher = aes.aes(int.from_bytes(ky_transpose, byteorder='big'), 128)
#Send key to Nano
target.simpleserial_write('p', ky)
target.simpleserial_read('r', 16)

Enter a 16 character Key1234567891234567
Set the Key to:  ['0x31', '0x35', '0x39', '0x34', '0x32', '0x36', '0x31', '0x35', '0x33', '0x37', '0x32', '0x36', '0x34', '0x38', '0x33', '0x37']


CWbytearray(b'31 32 33 34 35 36 37 38 39 31 32 33 34 35 36 37')

In [5]:
# ###After programming target, we will write several plain texts to it###
# ##First write the plain text, then get the encrypted text back###
# from tqdm import tnrange
# import aes
# import numpy as np
# import time

# #transposes a matrix 
# def transpose_matrix(lst):
#     matrix = [lst[i:i + 4] for i in range(0, len(lst), 4)]
#     transposed = [[matrix[j][i] for j in range(4)] for i in range(4)]
#     return [elem for row in transposed for elem in row]

# pt = bytearray([])
# ct = bytearray([])

# #for user input

# data = input("Enter a 16 character String")
# pt =bytearray(data.encode('utf-8'))

# #debug plaintext and ciphertext seen in FIPS test cases
# #pt = bytearray([0x32,0x88, 0x31, 0xe0, 0x43, 0x5a, 0x31, 0x37, 0xf6, 0x30, 0x98, 0x07, 0xa8, 0x8d, 0xa2, 0x34])
# #ky = bytearray([0x2b, 0x28, 0xab, 0x09, 0x7e, 0xae, 0xf7, 0xcf, 0x15, 0xd2, 0x15, 0x4f, 0x16, 0xa6, 0x88, 0x3c])

# #If the data is not 16 characters, pad it with zeros
# while len(pt) % 16 != 0:
#     pt.append(0)
# print("You entered: ", data)
# print("Input in Bytes: ",pt )
# #iterate 16 bytes at a time through the entire message 
# for i in range(0, len(pt), 16):
#     pt_chunk = pt[i:i+16]
#     print("Sending 16 bytes of Plaintext: ", pt_chunk)
#     target.simpleserial_write('p', pt_chunk)
#     #Read the ciphertext
#     response = target.simpleserial_read('r', 16)
#     ct[i:i+16] = transpose_matrix(response)
#     print("Recieved 16 bytes of CipherText: ", response)
#     #Decode the ciphertext
#     plaintext = cipher.dec_once(int.from_bytes(ct[i:i+16], byteorder='big'))
#     decrypted_bytes = [hex(num) for num in plaintext]
#     #Print the raw bytes and the ASCII Bytes
#     print("Decrytped Data: ", decrypted_bytes)
#     decrypted_bytes_transpose = transpose_matrix(decrypted_bytes)
#     print("Decrypted ASCII ", ''.join(chr(int(num, 16)) for num in decrypted_bytes_transpose))



    

In [8]:
from tqdm.notebook import trange
import numpy as np
import time
import aes

def capture_trace(_ignored=None):
    return cw.capture_trace(scope, target, text).wave

#transposes a matrix 
def transpose_matrix(lst):
    matrix = [lst[i:i + 4] for i in range(0, len(lst), 4)]
    transposed = [[matrix[j][i] for j in range(4)] for i in range(4)]
    return [elem for row in transposed for elem in row]


ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

proj = cw.create_project("RedTeamCPA")

N = 50
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next()
    
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    power_trace = cw.capture_trace(scope, target, text, key)
    
    target.simpleserial_read('r', 16)
    
    proj.traces.append(power_trace)
    
proj.save()

Capturing traces:   0%|          | 0/50 [00:00<?, ?it/s]

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 


In [9]:
import chipwhisperer.analyzer as cwa

leak_model = cwa.leakage_models.sbox_output

attack = cwa.cpa(proj, leak_model)

results = attack.run()

print(results)

Subkey KGuess Correlation
  00    0x2B    0.72382
  01    0x7E    0.74676
  02    0x15    0.69870
  03    0xDA    0.64749
  04    0x28    0.64401
  05    0xAE    0.70115
  06    0xD2    0.68835
  07    0x43    0.67188
  08    0x84    0.67066
  09    0x3D    0.70993
  10    0x15    0.69150
  11    0xB6    0.64006
  12    0x09    0.65243
  13    0xCF    0.68291
  14    0xC5    0.65656
  15    0x3C    0.66193



In [10]:
import pandas as pd
stat_data = results.find_maximums()
df = pd.DataFrame(stat_data).transpose()
print(df.head())

                                0                                1   \
0   [43, 4537, 0.7238219573153402]  [126, 4437, 0.7467580632729546]   
1  [226, 3487, 0.6482914821388299]  [112, 2820, 0.6387771223683449]   
2  [149, 2493, 0.6185278101571517]   [232, 3351, 0.616019939120609]   
3    [9, 4803, 0.6087649399432328]    [69, 450, 0.6074963866294704]   
4  [124, 4765, 0.5994594760718444]   [15, 4654, 0.6067348914020758]   

                                2                                3   \
0    [21, 4549, 0.698700381199398]  [218, 4747, 0.6474895729590099]   
1    [73, 510, 0.6266697731856606]   [117, 1150, 0.624960941161728]   
2  [239, 2026, 0.6172715397110945]    [8, 1691, 0.6143799785506897]   
3   [46, 2623, 0.6084215392301593]  [120, 2280, 0.6087567405879667]   
4   [15, 4934, 0.5845957067615007]   [154, 3976, 0.606979322033927]   

                                4                                5   \
0   [40, 4561, 0.6440109523218438]  [174, 4769, 0.7011544315764715]   
1   

In [ ]:
s